In [5]:
from pyspark.sql import SparkSession

try:
    spark
except NameError:
    spark = (
        SparkSession.builder
          .appName("nb-smoke")
          .master("spark://spark-master:7077")
          .config("spark.driver.bindAddress", "0.0.0.0")
          .config("spark.driver.host", "datalab-jupyter-notebook")
          .getOrCreate()
    )

print("Spark:", spark.version)


Spark: 3.5.6


In [6]:
from pyspark.sql import functions as F

df = spark.range(10).withColumn("x2", F.col("id") * 2)
df.show()

+---+---+
| id| x2|
+---+---+
|  0|  0|
|  1|  2|
|  2|  4|
|  3|  6|
|  4|  8|
|  5| 10|
|  6| 12|
|  7| 14|
|  8| 16|
|  9| 18|
+---+---+



In [7]:
big = spark.range(1_000_00)  # 100k linhas
res = big.repartition(4).agg(F.max("id").alias("max_id"), F.count("*").alias("rows"))
res.show()

+------+------+
|max_id|  rows|
+------+------+
| 99999|100000|
+------+------+



In [8]:
# opcional: limpa cache
spark.catalog.clearCache()

# se tiver queries de Structured Streaming ativas, pare-as antes:
for q in spark.streams.active:
    q.stop()

# para o Spark (derruba executors e libera recursos)
spark.stop()
